In [ ]:
# cd .\WorkingDirectory\LocalstackDockerCompose
# docker-compose up -d --build

In [ ]:
docker ps --filter="name=localstack_main"

[aws localstack and Terraform](https://docs.localstack.cloud/user-guide/integrations/terraform/)

In [ ]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$WorkingDirectory = $OriginalProjectPath + '\WorkingDirectory\LocalstackAwsExample'
"OriginalProjectPath: $OriginalProjectPath"
"WorkingDirectory: $WorkingDirectory"

In [ ]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName
"TerraformExePath: $TerraformExePath"

In [ ]:
.\terraform version

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory init -input=false"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory plan -input=false"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory apply --auto-approve"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = "awslocal dynamodb list-tables"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
if (!$IsWindows) {exit} # Guard Clause this section is only for Windows

In [ ]:
$command = @"
awslocal dynamodb create-table 
    --table-name Music 
    --attribute-definitions 
        AttributeName=Artist,AttributeType=S 
        AttributeName=SongTitle,AttributeType=S 
    --key-schema 
        AttributeName=Artist,KeyType=HASH 
        AttributeName=SongTitle,KeyType=RANGE 
    --provisioned-throughput 
        ReadCapacityUnits=5,WriteCapacityUnits=5 
    --table-class STANDARD
"@
Write-Output "Executing the command:"
$command = $command.replace("`n","").replace("`r","")
Write-Output $command
Invoke-Expression $command

In [ ]:
$command = "awslocal dynamodb scan --endpoint-url http://localhost:4566 --table-name Music"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:
# docker stop localstack_main